<a href="https://colab.research.google.com/github/KGL8/Code-for-Lab/blob/main/photon_through_deuteron_target.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [85]:
!pip install pythreejs

from google.colab import output
output.enable_custom_widget_manager()
import numpy as np
from pythreejs import *
import ipywidgets as widgets

In [86]:
# create an instance of the line segment object
def ray(origin, direction, length, col):
    return Line(geometry=BufferGeometry(attributes={'position': BufferAttribute(array=np.array([origin, length * np.array(direction)]), normalized=False),}),material=LineBasicMaterial(color=col))

In [87]:
# create an instance of the box object
def point(position, col, size):
    return Points(geometry=BufferGeometry(attributes={"position": BufferAttribute(np.array([position]), normalized=False)}),
                  material=PointsMaterial(color=col, size=size, sizeAttenuation=False))

In [88]:
# create an instance of the box object
def box(width, height, depth, position, col):
    box_geometry = BoxBufferGeometry(width=width, height=height, depth=depth)

    box_material = MeshPhysicalMaterial(color=col, transparent=True, opacity=0.5)
    box_mesh = Mesh(geometry=box_geometry, material=box_material)

    wireframe_material = LineBasicMaterial(color='#ff7070',linewidth=2)
    wireframe = WireframeGeometry(box_geometry)
    wireframe_mesh = LineSegments(wireframe, wireframe_material)

    box_group = Group(children=[box_mesh, wireframe_mesh])
    box_group.position = position

    return box_group

In [89]:
# cartesian to spherical
def car_sph(x,y,z):
  r = np.sqrt(x**2 + y**2 + z**2)
  theta = np.arccos(z / r)
  phi = np.arctan2(y, x)
  return np.array([theta, phi, r])

In [90]:
# spherical to cartesian
def sph_car(theta, phi, r=1):
    x = r * np.sin(theta) * np.cos(phi)
    y = r * np.sin(theta) * np.sin(phi)
    z = r * np.cos(theta)
    return np.array([x, y, z])

In [91]:
# entry and exit points for a ray through an axis-aligned box
def intersect_ray_box(ray_origin, ray_direction, box_min, box_max):
    inv_dir = 1 / np.where(ray_direction != 0, ray_direction, np.inf)
    tmin = (box_min - ray_origin) * inv_dir
    tmax = (box_max - ray_origin) * inv_dir

    t_enter = np.max(np.minimum(tmin, tmax))
    t_exit = np.min(np.maximum(tmin, tmax))

    if t_enter > t_exit or t_exit < 0:
        return None, None

    return ray_origin + t_enter * ray_direction, ray_origin + t_exit * ray_direction

In [92]:
# parameters
origin = 3
width = 2
height = 5
depth = 7
box_min = np.array([-width / 2, -height / 2, origin])
box_max = np.array([width / 2, height / 2, depth + origin])
dir = np.array([0.3,-0.1,1])
en,ex = intersect_ray_box(np.array([0, 0, 0]), dir, box_min, box_max)

In [93]:
# distance the photon travels through the Deuterium target
if en is not None and ex is not None:
    print("Distance:", np.linalg.norm(en - ex))
else:
    print("No intersection found")

Distance: 0.3496029493900507


In [94]:
# UI rendering
target_box = box(width, height, depth, (0, 0, depth / 2 + origin), 'red')


ray_line = ray(np.array([0, 0, 0]), dir, 1000, '#f5f542')

z_ray = ray(np.array([0, 0, 0]), sph_car(0, 0), 1, 'red')
y_ray = ray(np.array([0, 0, 0]), sph_car(np.pi/2, np.pi/2), 1, 'green')
x_ray = ray(np.array([0, 0, 0]), sph_car(np.pi/2, 0), 1, 'blue')

origin_point = point([0, 0, 0], 'white', 4)

scene = Scene(children=[target_box, ray_line, origin_point, x_ray, y_ray, z_ray])
scene.background = '#141414'

if en is not None and ex is not None:
  entry = point(en, 'blue', 7)
  exit = point(ex, 'blue', 7)
  scene.add(entry)
  scene.add(exit)

lights = [
    DirectionalLight(color='white', intensity=3, position=[0, 200, 0]),
    DirectionalLight(color='white', intensity=3, position=[100, 200, 100]),
    DirectionalLight(color='white', intensity=3, position=[-100, -200, -100])
]
for light in lights:
    scene.add(light)

camera = PerspectiveCamera(position=[10, 10, 10], up=[0, 1, 0], children=[DirectionalLight(color='white', position=[3, 5, 1], intensity=0.5)])

display(Renderer(camera=camera, scene=scene, controls=[OrbitControls(controlling=camera)], antialias=True, width=800, height=600))

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.5, position=(3.0, 5.0,…

TraitError: The 'target' trait of a DirectionalLight instance expected an Uninitialized or an Object3D, not the str 'IPY_MODEL_267ae9d8-f065-4bf1-923c-61f985034d79'.

TraitError: The 'target' trait of a DirectionalLight instance expected an Uninitialized or an Object3D, not the str 'IPY_MODEL_edd439ee-33fb-41c6-97d6-559134546590'.

TraitError: The 'target' trait of a DirectionalLight instance expected an Uninitialized or an Object3D, not the str 'IPY_MODEL_5e833b15-97c5-431d-98bb-1fa96898c1fb'.

TraitError: The 'target' trait of a DirectionalLight instance expected an Uninitialized or an Object3D, not the str 'IPY_MODEL_f4071079-caa9-407a-86aa-a09e860c6f8b'.